In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
train_data = pd.read_excel(r'/content/drive/MyDrive/EEG_Dataset_Decoded/Training_data.xlsx',header=None)

In [ ]:
print(train_data.shape)
print("Number of rows: "+str(train_data.shape[0]))
print("Number of columns: "+str(train_data.shape[1]))

In [ ]:
train_labels = pd.read_excel(r'/content/drive/MyDrive/EEG_Dataset_Decoded/Training_labels.xlsx',header=None)

In [ ]:
print(train_labels.shape)
print("Number of rows: "+str(train_labels.shape[0]))
print("Number of columns: "+str(train_labels.shape[1]))

#Exploratory Data Analysis

In [ ]:
train_labels.value_counts() # To find the dataset is balanced or not: In this case it is balanced

In [ ]:
train_labels.isnull().values.any() # Machine Learning models will not accept null values. In this dataset there are no null values.

In [ ]:
train_data.head()

# Data Preprocessing

In [ ]:
# define min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler_train = MinMaxScaler()
# transform data
scaled_train_data = scaler_train.fit_transform(train_data)
print(scaled_train_data)

In [ ]:
# define min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler_test = MinMaxScaler()
# transform data
scaled_test_data = scaler_train.fit_transform(test_data)
print(scaled_test_data)

In [ ]:
scaled_train_data

#Dimensionality Reduction

In [ ]:
#PCA
from sklearn.decomposition import PCA
import time

In [ ]:
# Train Data Dimensionality reduction
%%time
start = time.time()
# Program to find the optimal number of components for PCA
n_comp1 = [2,3,4,10,15,20,25, 40, 50, 75, 100,150,200,500] # list containing different values of components
explained1 = [] # explained variance ratio for each component of PCA
for x1 in n_comp1:
    pca = PCA(n_components=x1)
    pca.fit(scaled_train_data)
    explained1.append(pca.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x1,pca.explained_variance_ratio_.sum()))
plt.plot(n_comp1, explained1)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()
print('Duration: {} seconds'.format(time.time() - start))

In [ ]:
start = time.time()
pca = PCA(n_components=25)
principalComponents_train = pca.fit_transform(scaled_train_data)
print('Duration: {} seconds'.format(time.time() - start))
train_pca = pd.DataFrame(data = principalComponents_train)
#              , columns = ['principal component 1', 'principal component 2','principal component 3'])

In [ ]:
train_pca

In [ ]:
start = time.time()
pca = PCA(n_components=25)
principalComponents_test = pca.fit_transform(scaled_test_data)
print('Duration: {} seconds'.format(time.time() - start))
test_pca = pd.DataFrame(data = principalComponents_test)
#              , columns = ['principal component 1', 'principal component 2','principal component 3'])

In [ ]:
test_pca

In [ ]:
train_out = train_labels[0].ravel()

In [ ]:
train_labels.info()

In [ ]:
train_out

# Train and Test Split for ML Model Creation

In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(train_pca, train_out, test_size = 0.20, random_state = 100)

In [ ]:
from sklearn.ensemble import RandomForestClassifier # Non-Linear Classifier
from sklearn import svm # Linear Classifier
classifier = RandomForestClassifier() # Classifier is instantiated
classifier.fit(X_Train,Y_Train) # Model Training

In [ ]:
Y_Pred = classifier.predict(X_Test) # Test Testing

In [ ]:
import seaborn as sns
from sklearn import metrics 
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
matrix = confusion_matrix(Y_Test, Y_Pred) # Y-test = True Value,Y_Pred = Predicting Values 
sns.heatmap(matrix, annot=True, fmt="d")
plt.title('Confusion Matrix_RF')
plt.xlabel('Predicted')
plt.ylabel('True')
print(classification_report(Y_Test, Y_Pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
classifier_svc = svm.SVC()
classifier_svc.fit(X_Train,Y_Train)

In [ ]:
Y_Pred_svc = classifier_svc.predict(X_Test)

In [ ]:
import seaborn as sns
from sklearn import metrics 
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
matrix = confusion_matrix(Y_Test, Y_Pred_svc)
sns.heatmap(matrix, annot=True, fmt="d")
plt.title('Confusion Matrix_SVC')
plt.xlabel('Predicted')
plt.ylabel('True')
print(classification_report(Y_Test, Y_Pred_svc))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import XGBClassifier
from xgboost import XGBClassifier

In [ ]:
# declare parameters
params = {
            'objective':'multi:softprob',
            'max_depth': 50,
            'alpha': 10,
            'learning_rate': 0.1,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
# xgb_clf = XGBClassifier()

In [ ]:
xgb_clf.fit(X_Train,Y_Train)

In [ ]:
Y_Pred_xgb = xgb_clf.predict(X_Test)

In [ ]:
import seaborn as sns
from sklearn import metrics 
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
matrix = confusion_matrix(Y_Test, Y_Pred_xgb)
sns.heatmap(matrix, annot=True, fmt="d")
plt.title('Confusion Matrix_XGB')
plt.xlabel('Predicted')
plt.ylabel('True')
print(classification_report(Y_Test, Y_Pred_xgb))